# Chapter 3 -- NumPy and Pandas
## *Python for AI/ML: A Complete Learning Journey*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/CH03_NumPy_and_Pandas.ipynb)
&nbsp;&nbsp;[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)

---

**Part:** 2 -- Data Science Foundations  
**Prerequisites:** Chapter 2 (Intermediate Python)  
**Estimated time:** 5-6 hours

---

### Learning Objectives

By the end of this chapter you will be able to:

- Create and manipulate NumPy arrays with indexing, slicing, and broadcasting
- Use NumPy universal functions (ufuncs) and random number generation
- Understand why NumPy is 100x faster than pure Python for numerical work
- Create Pandas Series and DataFrames from multiple sources
- Select data with `.loc`, `.iloc`, and boolean indexing
- Clean real-world data: missing values, duplicates, dtype conversion
- Aggregate and group data with `groupby`, `agg`, and `pivot_table`
- Apply functions efficiently with `apply()` vs vectorised operations
- Merge and join DataFrames

---

### Project Thread -- Chapter 3

This is the chapter where the SO 2025 dataset becomes real.
We load the full 15,000-row curated subset, clean it end-to-end,
and build the analysis-ready DataFrame that every subsequent chapter uses.
By the end of this chapter you will have handled salary outliers, exploded
multi-value language columns, and produced a cleaned dataset ready for
visualisation in Chapter 4 and modelling in Chapter 6.


---

## Setup -- Imports and Version Check


In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')   # suppress minor deprecation warnings

print(f'NumPy:  {np.__version__}')
print(f'Pandas: {pd.__version__}')

DATASET_URL = 'https://raw.githubusercontent.com/timothy-watt/python-for-ai-ml/main/data/so_survey_2025_curated.csv'


---

## Section 3.1 -- NumPy: Fast Numerical Computing

NumPy (Numerical Python) is the foundation of the entire Python data science stack.
Pandas, scikit-learn, PyTorch, and TensorFlow all store data as NumPy arrays
internally -- or arrays with a compatible interface.

The key insight: a NumPy array stores all its data in a single contiguous block
of memory with a fixed type. This lets C code loop over it at full CPU speed --
100-1000x faster than a Python list, which stores pointers to individual Python objects.


In [ ]:
# 3.1.1 -- Creating arrays and understanding dtypes

# From a Python list -- NumPy infers the dtype automatically
salaries_list = [72_000, 88_000, 105_000, 135_000, 155_000, 195_000, 210_000]
a = np.array(salaries_list)
print(f'Array:  {a}')
print(f'dtype:  {a.dtype}')    # int64 -- 64-bit integer
print(f'shape:  {a.shape}')    # (7,) -- 1D array with 7 elements
print(f'ndim:   {a.ndim}')     # 1 -- number of dimensions
print(f'size:   {a.size}')     # 7 -- total number of elements

# Force a specific dtype
a_float = np.array(salaries_list, dtype=np.float64)
print(f'float64 array: {a_float}')

# 2D array -- a matrix
features = np.array([
    [7,  135_000, 1],   # [years_exp, salary, uses_python]
    [3,   88_000, 0],
    [9,  105_000, 1],
    [12, 210_000, 1],
], dtype=np.float64)
print(f'2D shape: {features.shape}')   # (4, 3) -- 4 rows, 3 columns

# Array creation shortcuts
zeros  = np.zeros((3, 4))          # 3x4 matrix of 0.0
ones   = np.ones((2, 5))           # 2x5 matrix of 1.0
eye    = np.eye(4)                 # 4x4 identity matrix
rng    = np.arange(0, 100, 10)     # like range() but returns an array
linsp  = np.linspace(0, 1, 11)     # 11 evenly spaced values from 0 to 1

print(f'zeros shape:  {zeros.shape}')
print(f'arange:       {rng}')
print(f'linspace:     {linsp}')


In [ ]:
# 3.1.2 -- Indexing, slicing, and boolean masking

a = np.array([72_000, 88_000, 105_000, 135_000, 155_000, 195_000, 210_000])

# 1D indexing -- identical to Python lists
print(f'First:    {a[0]}')
print(f'Last:     {a[-1]}')
print(f'Slice:    {a[2:5]}')
print(f'Reversed: {a[::-1]}')

# 2D indexing -- [row, column]
m = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(f'Element [1,2]: {m[1, 2]}')    # row 1, col 2 -> 6
print(f'Row 0:         {m[0, :]}')    # all columns of row 0
print(f'Col 1:         {m[:, 1]}')    # all rows of column 1
print(f'Submatrix:     {m[0:2, 1:3]}')  # rows 0-1, cols 1-2

# Boolean masking -- the backbone of data filtering
# Step 1: create a boolean array where condition is True
mask = a > 100_000
print(f'Mask:          {mask}')        # [False False  True  True  True  True  True]

# Step 2: use the mask to select elements
high_salaries = a[mask]
print(f'High salaries: {high_salaries}')

# Combine conditions with & (and) and | (or) -- NOT Python's 'and'/'or'
middle = a[(a >= 100_000) & (a <= 160_000)]
print(f'Middle band:   {middle}')

# np.where -- like a vectorised if/else
labels = np.where(a >= 150_000, 'Senior', 'Junior')
print(f'Labels:        {labels}')


In [ ]:
# 3.1.3 -- Broadcasting
#
# Broadcasting is NumPy's rule for performing arithmetic between arrays
# of different shapes. It eliminates most explicit loops over array elements.
# Rule: dimensions are compatible if they are equal OR one of them is 1.

salaries = np.array([72_000, 88_000, 105_000, 135_000, 210_000], dtype=float)

# Scalar broadcast -- the scalar is applied to every element
print('--- Scalar operations ---')
print(f'Original:     {salaries}')
print(f'+10% raise:   {salaries * 1.1}')       # multiplies every element
print(f'- $5k:        {salaries - 5_000}')      # subtracts from every element
print(f'Normalised:   {(salaries - salaries.mean()) / salaries.std()}')

# Why print .shape before operations -- a debugging habit
a = np.array([[1, 2, 3], [4, 5, 6]])     # shape (2, 3)
b = np.array([10, 20, 30])               # shape (3,) -- broadcast across rows
print(f'a shape: {a.shape},  b shape: {b.shape}')
print(f'a + b:\n{a + b}')   # b is added to each row of a

# Column broadcast -- reshape b to (2,1) to broadcast across columns
c = np.array([[100], [200]])             # shape (2, 1)
print(f'a shape: {a.shape},  c shape: {c.shape}')
print(f'a + c:\n{a + c}')   # c is added to each column of a

# Real use: subtract the column mean from each column (feature centering)
data = np.array([[72_000, 7], [88_000, 3], [105_000, 9], [135_000, 12]], dtype=float)
col_means = data.mean(axis=0)   # axis=0 = compute along rows (per column)
centered  = data - col_means    # broadcasts col_means across all rows
print(f'Column means: {col_means}')
print(f'Centered data:\n{centered}')


In [ ]:
# 3.1.4 -- Universal functions (ufuncs) and aggregations

salaries = np.array([72_000, 88_000, 105_000, 135_000, 155_000, 195_000, 210_000], dtype=float)

# Aggregation functions -- collapse an array to a scalar
print('--- Aggregations ---')
print(f'sum:    ${np.sum(salaries):>12,.0f}')
print(f'mean:   ${np.mean(salaries):>12,.2f}')
print(f'median: ${np.median(salaries):>12,.2f}')
print(f'std:    ${np.std(salaries):>12,.2f}')
print(f'min:    ${np.min(salaries):>12,.0f}')
print(f'max:    ${np.max(salaries):>12,.0f}')
print(f'argmax: index {np.argmax(salaries)} (highest earner)')

# Math ufuncs -- element-wise operations
print()
print('--- Math ufuncs ---')
log_sals = np.log(salaries)    # natural log -- common for skewed salary distributions
print(f'log(salaries): {np.round(log_sals, 2)}')

# 2D aggregations -- axis parameter controls direction
matrix = np.array([[72_000, 7, 1],
                   [88_000, 3, 0],
                   [105_000, 9, 1],
                   [135_000, 12, 1]], dtype=float)

print()
print('--- 2D aggregations ---')
print(f'Column means (axis=0): {matrix.mean(axis=0)}')   # mean of each column
print(f'Row means    (axis=1): {matrix.mean(axis=1)}')   # mean of each row

# Random number generation -- essential for ML (weight initialisation, train/test splits)
rng = np.random.default_rng(seed=42)   # new-style: reproducible random generator
print()
print('--- Random ---')
print(f'Uniform [0,1]:    {rng.random(5).round(4)}')
print(f'Normal (mu=0):    {rng.standard_normal(5).round(4)}')
print(f'Integers 0-99:    {rng.integers(0, 100, size=5)}')
random_salaries = rng.normal(loc=120_000, scale=30_000, size=1000)
print(f'Simulated salaries: mean=${random_salaries.mean():,.0f}, std=${random_salaries.std():,.0f}')


In [ ]:
# 3.1.5 -- Speed comparison: NumPy vs pure Python
import time

n = 1_000_000   # one million elements

# Pure Python: list of numbers
py_list = list(range(n))
start = time.perf_counter()
py_result = sum(x * 1.1 for x in py_list)
py_time = time.perf_counter() - start

# NumPy: array of numbers
np_array = np.arange(n, dtype=float)
start = time.perf_counter()
np_result = (np_array * 1.1).sum()
np_time = time.perf_counter() - start

print(f'Pure Python: {py_time*1000:>8.2f} ms')
print(f'NumPy:       {np_time*1000:>8.2f} ms')
print(f'Speedup:     {py_time/np_time:.0f}x faster with NumPy')
print(f'Results match: {abs(py_result - np_result) < 1}')

# Memory comparison
import sys
py_mem = sys.getsizeof(py_list) + sum(sys.getsizeof(x) for x in py_list[:100]) * 10
np_mem = np_array.nbytes
print(f'Python list memory: ~{py_mem/1e6:.0f} MB (estimated)')
print(f'NumPy array memory:  {np_mem/1e6:.1f} MB')


In [ ]:
# 3.1.6 -- Linear algebra basics (preview for Chapter 7)
#
# Neural networks are fundamentally matrix multiplications.
# Here we cover the operations Chapter 7 will rely on.

# Matrix multiplication -- the @ operator
# In ML: output = inputs @ weights + bias
inputs  = np.array([[1.0, 2.0, 3.0],
                    [4.0, 5.0, 6.0]])   # shape (2, 3): 2 samples, 3 features
weights = np.array([[0.1, 0.2],
                    [0.3, 0.4],
                    [0.5, 0.6]])         # shape (3, 2): 3 inputs, 2 outputs
bias    = np.array([0.01, 0.02])         # shape (2,): one bias per output

output = inputs @ weights + bias         # shape (2, 2)
print(f'inputs shape:  {inputs.shape}')
print(f'weights shape: {weights.shape}')
print(f'output shape:  {output.shape}')
print(f'output:\n{output}')

# Transpose
print(f'weights.T shape: {weights.T.shape}')

# Dot product -- similarity between two vectors
v1 = np.array([1.0, 0.0, 0.0])   # unit vector
v2 = np.array([0.8, 0.6, 0.0])   # similar direction
v3 = np.array([0.0, 1.0, 0.0])   # orthogonal
print(f'dot(v1, v2) = {np.dot(v1, v2):.2f}  (similar: high)')
print(f'dot(v1, v3) = {np.dot(v1, v3):.2f}  (orthogonal: zero)')

# Norm -- magnitude of a vector
print(f'norm(v2) = {np.linalg.norm(v2):.4f}')   # should be 1.0 for unit vector


---

## Section 3.2 -- Pandas: DataFrames for Data Science

Pandas is to tabular data what NumPy is to numerical arrays.
A **DataFrame** is a 2D table where each column can have its own dtype --
like a spreadsheet that lives in memory and has hundreds of analysis methods.

The two core Pandas data structures:
- **Series** -- a 1D labelled array (one column, or one row)
- **DataFrame** -- a 2D labelled table (rows and columns)

Every column in a DataFrame is a Series. Every row is also accessible as a Series.


In [ ]:
# 3.2.1 -- Series: the building block

# Create a Series from a list -- index is auto-assigned 0, 1, 2...
salaries = pd.Series([72_000, 88_000, 105_000, 135_000, 210_000])
print('Basic Series:')
print(salaries)
print(f'dtype: {salaries.dtype}')

# Create with a custom index -- index can be any hashable values
sal_named = pd.Series(
    [72_000, 88_000, 105_000, 135_000, 210_000],
    index=['R001', 'R002', 'R003', 'R004', 'R005'],
    name='salary'
)
print()
print('Named Series with custom index:')
print(sal_named)

# Series arithmetic -- broadcasts like NumPy
print(f'Mean:   ${sal_named.mean():,.0f}')
print(f'Median: ${sal_named.median():,.0f}')
print(f'Above mean: {(sal_named > sal_named.mean()).sum()} respondents')

# Series from dict -- keys become the index
country_counts = pd.Series({'USA': 4521, 'Germany': 1832, 'UK': 2104, 'India': 3210})
print()
print('Country counts:')
print(country_counts.sort_values(ascending=False))


In [ ]:
# 3.2.2 -- Creating DataFrames

# From a list of dicts -- the most common way in practice
data = [
    {'id': 'R001', 'country': 'USA',     'salary': 135_000, 'years_exp': 7,  'lang': 'Python'},
    {'id': 'R002', 'country': 'UK',      'salary':  88_000, 'years_exp': 3,  'lang': 'JavaScript'},
    {'id': 'R003', 'country': 'Germany', 'salary': 105_000, 'years_exp': 9,  'lang': 'Python'},
    {'id': 'R004', 'country': 'India',   'salary':  42_000, 'years_exp': 2,  'lang': 'Java'},
    {'id': 'R005', 'country': 'Canada',  'salary': 118_000, 'years_exp': 11, 'lang': 'Python'},
]
df = pd.DataFrame(data)
print('DataFrame from list of dicts:')
print(df)
print(f'Shape: {df.shape}   dtype per column:')
print(df.dtypes)

# From a dict of lists -- columns as keys
df2 = pd.DataFrame({
    'salary':    [72_000, 88_000, 135_000],
    'years_exp': [3, 5, 9],
    'python':    [True, False, True],
})
print()
print('DataFrame from dict of lists:')
print(df2)

# From a NumPy array
arr = np.array([[72_000, 7], [88_000, 3], [135_000, 9]])
df3 = pd.DataFrame(arr, columns=['salary', 'years_exp'])
print()
print('DataFrame from NumPy array:')
print(df3)


In [ ]:
# 3.2.3 -- Selection: .loc, .iloc, and boolean indexing
#
# This is the most important section to get right -- confusing loc/iloc
# causes silent data errors that are hard to debug.
#
#   df[col]          -- select a column by name (returns Series)
#   df[[c1, c2]]     -- select multiple columns (returns DataFrame)
#   df.loc[row, col] -- select by LABEL (index value and column name)
#   df.iloc[r, c]    -- select by INTEGER POSITION (0-based)

df = pd.DataFrame(data)   # reuse the 5-row DataFrame from above
df = df.set_index('id')   # use 'id' as the row index

print('--- Column selection ---')
print(df['salary'])                    # single column -> Series
print()
print(df[['salary', 'country']])       # multiple columns -> DataFrame

print()
print('--- .loc (label-based) ---')
print(df.loc['R001'])                  # one row by index label
print()
print(df.loc['R001', 'salary'])        # specific cell: row label, col name
print(df.loc[['R001', 'R003'], ['salary', 'years_exp']])  # multiple rows and cols

print()
print('--- .iloc (position-based) ---')
print(df.iloc[0])                      # first row by position
print(df.iloc[0, 1])                   # row 0, col 1
print(df.iloc[0:3, 0:2])              # first 3 rows, first 2 cols

print()
print('--- Boolean indexing ---')
high_earners = df[df['salary'] > 100_000]
print(high_earners)

python_devs = df[(df['lang'] == 'Python') & (df['years_exp'] >= 5)]
print()
print('Python devs with 5+ years:')
print(python_devs)


### 3.2.4 -- Project: Loading the Full SO 2025 Dataset

This is the moment the book's dataset becomes fully operational.
We load all 15,000 rows, run the three essential first commands on every
new dataset, and understand the shape of what we are working with.


In [ ]:
# Load the full SO 2025 curated dataset
print(f'Loading: {DATASET_URL}')
df = pd.read_csv(DATASET_URL)

# The three commands you run on every new dataset -- in this order
print(f'Shape: {df.shape[0]:,} rows x {df.shape[1]} columns')
print()
print('First 3 rows:')
print(df.head(3).to_string())


In [ ]:
# .info() -- the single most useful first look at any DataFrame
# Shows: column names, non-null counts, dtypes, memory usage
print('DataFrame Info:')
df.info()


In [ ]:
# .describe() -- statistical summary of numeric columns
print('Numeric column statistics:')
df.describe().round(0)


---

## Section 3.3 -- Data Cleaning

Real-world data is always messy. The SO 2025 dataset is no exception:
it has missing values, salary outliers spanning from $1 to $20M+,
multi-value columns where one cell contains a semicolon-separated list,
and columns that need type conversion.

Cleaning is not glamorous, but it is the difference between a model that
trains on corrupted data and one that produces trustworthy predictions.
> **Rule:** Never modify `df` in-place during cleaning without keeping a backup.
> Use `df.copy()` to create a clean working copy.


In [ ]:
# 3.3.1 -- Auditing missing values

# df.isnull() returns a boolean DataFrame: True where data is missing
missing = df.isnull().sum().sort_values(ascending=False)
missing_pct = (missing / len(df) * 100).round(1)

missing_report = pd.DataFrame({
    'missing_count': missing,
    'missing_pct': missing_pct
}).query('missing_count > 0')   # only rows with at least one missing value

print(f'Columns with missing values: {len(missing_report)} of {df.shape[1]}')
print()
print(missing_report.to_string())


In [ ]:
# 3.3.2 -- Handling missing values: drop vs fill

df_clean = df.copy()   # always work on a copy

# Strategy 1: drop rows where the target column (salary) is missing
# We CANNOT impute the target variable -- those rows must go
before = len(df_clean)
df_clean = df_clean.dropna(subset=['ConvertedCompYearly'])
print(f'Dropped {before - len(df_clean):,} rows with missing salary')
print(f'Remaining: {len(df_clean):,} rows')

# Strategy 2: fill missing categorical columns with a placeholder
cat_cols = ['Country', 'DevType', 'EdLevel', 'Employment', 'RemoteWork', 'OrgSize']
for col in cat_cols:
    if col in df_clean.columns:
        n_filled = df_clean[col].isnull().sum()
        df_clean[col] = df_clean[col].fillna('Unknown')
        if n_filled > 0:
            print(f'Filled {n_filled:,} missing values in {col} with "Unknown"')

# Strategy 3: fill numeric columns with median (robust to outliers)
if 'YearsCodePro' in df_clean.columns:
    # YearsCodePro has entries like 'Less than 1 year' and 'More than 50 years'
    # Convert to numeric first, coercing non-numeric strings to NaN
    df_clean['YearsCodePro'] = pd.to_numeric(df_clean['YearsCodePro'], errors='coerce')
    median_yrs = df_clean['YearsCodePro'].median()
    n_filled   = df_clean['YearsCodePro'].isnull().sum()
    df_clean['YearsCodePro'] = df_clean['YearsCodePro'].fillna(median_yrs)
    print(f'Filled {n_filled:,} missing YearsCodePro with median ({median_yrs:.0f} years)')

print(f'Missing values remaining: {df_clean.isnull().sum().sum()}')


In [ ]:
# 3.3.3 -- Salary outlier handling
#
# ConvertedCompYearly ranges from $1 (data entry error) to $20M+ (noise).
# We use the IQR (interquartile range) method to identify outlier bounds,
# then apply a sensible hard cap that preserves real high earners.

sal = df_clean['ConvertedCompYearly']

print('Salary distribution BEFORE cleaning:')
print(f'  Count:  {sal.count():,}')
print(f'  Min:    ${sal.min():,.0f}')
print(f'  p1:     ${sal.quantile(0.01):,.0f}')
print(f'  Median: ${sal.median():,.0f}')
print(f'  Mean:   ${sal.mean():,.0f}')
print(f'  p99:    ${sal.quantile(0.99):,.0f}')
print(f'  Max:    ${sal.max():,.0f}')

# IQR bounds
Q1  = sal.quantile(0.25)
Q3  = sal.quantile(0.75)
IQR = Q3 - Q1
lower_bound = max(Q1 - 3 * IQR, 5_000)      # at least $5k
upper_bound = min(Q3 + 3 * IQR, 600_000)    # cap at $600k

print(f'IQR bounds: ${lower_bound:,.0f} to ${upper_bound:,.0f}')

before = len(df_clean)
df_clean = df_clean[
    (df_clean['ConvertedCompYearly'] >= lower_bound) &
    (df_clean['ConvertedCompYearly'] <= upper_bound)
]
print(f'Removed {before - len(df_clean):,} outlier salary rows')

print()
print('Salary distribution AFTER cleaning:')
sal_clean = df_clean['ConvertedCompYearly']
print(f'  Count:  {sal_clean.count():,}')
print(f'  Min:    ${sal_clean.min():,.0f}')
print(f'  Median: ${sal_clean.median():,.0f}')
print(f'  Mean:   ${sal_clean.mean():,.0f}')
print(f'  Max:    ${sal_clean.max():,.0f}')


In [ ]:
# 3.3.4 -- Handling multi-value columns
#
# LanguageHaveWorkedWith stores multiple values per row: 'Python;SQL;JavaScript'
# To analyse language popularity, we need to 'explode' this column.
# This converts one row with 3 languages into 3 rows with 1 language each.

lang_col = 'LanguageHaveWorkedWith'

if lang_col in df_clean.columns:
    # Step 1: split the semicolon-separated string into a list
    # str accessor: vectorised string methods for the whole column
    lang_lists = df_clean[lang_col].str.split(';')
    print(f'Example before split: {df_clean[lang_col].iloc[0]}')
    print(f'Example after split:  {lang_lists.iloc[0]}')

    # Step 2: explode -- one row per language
    lang_exploded = lang_lists.explode()   # one language per row
    lang_exploded = lang_exploded.str.strip()   # remove any whitespace
    lang_exploded = lang_exploded[lang_exploded.notna() & (lang_exploded != '')]  # drop empties

    # Step 3: count and rank
    lang_counts = lang_exploded.value_counts()

    print(f'Unique languages in dataset: {len(lang_counts)}')
    print()
    print('Top 15 languages by respondent count:')
    top15 = lang_counts.head(15)
    for rank, (lang, count) in enumerate(top15.items(), 1):
        pct = count / len(df_clean) * 100
        bar = '#' * int(pct / 1.5)
        print(f'  #{rank:>2}: {lang:<25} {count:>5,}  ({pct:>5.1f}%)  {bar}')

    # Step 4: add a convenience column to df_clean
    # Boolean column: does this respondent use Python?
    df_clean['uses_python'] = df_clean[lang_col].str.contains('Python', na=False)
    python_pct = df_clean['uses_python'].mean() * 100
    print(f'Respondents using Python: {python_pct:.1f}%')


In [ ]:
# 3.3.5 -- Dtype conversion and final clean-up

# Check current dtypes
print('Current dtypes:')
print(df_clean.dtypes)
print()

# Convert object columns that should be categories
# 'category' dtype uses less memory for columns with few unique values
cat_cols = ['Country', 'EdLevel', 'Employment', 'RemoteWork', 'OrgSize']
for col in cat_cols:
    if col in df_clean.columns:
        n_unique = df_clean[col].nunique()
        df_clean[col] = df_clean[col].astype('category')
        print(f'  {col:<20} -> category  ({n_unique} unique values)')

# Reset index after all the row-dropping operations
df_clean = df_clean.reset_index(drop=True)

# Final shape
print()
print('=== Final cleaned dataset ===')
print(f'Shape: {df_clean.shape[0]:,} rows x {df_clean.shape[1]} columns')
print(f'Memory: {df_clean.memory_usage(deep=True).sum() / 1e6:.1f} MB')
print(f'Missing values: {df_clean.isnull().sum().sum()}')


---

## Section 3.4 -- Exploration and Aggregation

With a clean dataset, we can now ask real questions about it.
Pandas `groupby` and `agg` are the primary tools for computing
group-level statistics -- the equivalent of SQL's `GROUP BY`.


In [ ]:
# 3.4.1 -- groupby basics

# groupby splits the DataFrame into groups, applies a function to each,
# then combines the results -- the Split-Apply-Combine pattern.

if 'RemoteWork' in df_clean.columns:
    # Mean salary by remote work status
    remote_salary = (
        df_clean.groupby('RemoteWork')['ConvertedCompYearly']
        .agg(['mean', 'median', 'count'])
        .round(0)
        .sort_values('median', ascending=False)
    )
    remote_salary.columns = ['mean_salary', 'median_salary', 'count']
    print('Salary by remote work status:')
    print(remote_salary.to_string())

# Multi-column groupby
if 'EdLevel' in df_clean.columns and 'RemoteWork' in df_clean.columns:
    top_ed = df_clean['EdLevel'].value_counts().head(3).index.tolist()
    top_remote = df_clean['RemoteWork'].value_counts().head(3).index.tolist()
    subset = df_clean[
        df_clean['EdLevel'].isin(top_ed) & df_clean['RemoteWork'].isin(top_remote)
    ]
    cross = (
        subset.groupby(['EdLevel', 'RemoteWork'])['ConvertedCompYearly']
        .median()
        .unstack()     # pivot RemoteWork values into columns
        .round(0)
    )
    print()
    print('Median salary: Education x Remote Work')
    print(cross.to_string())


In [ ]:
# 3.4.2 -- .agg() with multiple functions and custom aggregations

# Top 8 countries by respondent count
top_countries = df_clean['Country'].value_counts().head(8).index.tolist()
df_top = df_clean[df_clean['Country'].isin(top_countries)]

# Apply multiple aggregation functions at once with a dict
country_stats = df_top.groupby('Country')['ConvertedCompYearly'].agg(
    count    = 'count',
    mean     = 'mean',
    median   = 'median',
    std      = 'std',
    p25      = lambda x: x.quantile(0.25),
    p75      = lambda x: x.quantile(0.75),
).round(0).sort_values('median', ascending=False)

print('Salary statistics by country (top 8):')
print(country_stats.to_string())


In [ ]:
# 3.4.3 -- apply() vs vectorised operations: when to use each
#
# apply() calls a Python function on each row or column.
# Vectorised operations use Pandas/NumPy built-ins on the whole column.
# Vectorised is ALWAYS faster -- use apply() only when no vectorised option exists.

import time

sal = df_clean['ConvertedCompYearly']

# Task: add a salary band label column

# Method 1: apply() with a Python function
def band_label(s):
    if s >= 200_000: return 'Principal'
    if s >= 150_000: return 'Senior+'
    if s >= 100_000: return 'Senior'
    if s >=  60_000: return 'Mid-level'
    return 'Junior'

start = time.perf_counter()
bands_apply = sal.apply(band_label)
t_apply = time.perf_counter() - start

# Method 2: pd.cut() -- vectorised binning (no Python loop)
start = time.perf_counter()
bands_cut = pd.cut(
    sal,
    bins=[0, 60_000, 100_000, 150_000, 200_000, float('inf')],
    labels=['Junior', 'Mid-level', 'Senior', 'Senior+', 'Principal'],
    right=False
)
t_cut = time.perf_counter() - start

print(f'apply() time:   {t_apply*1000:.2f} ms')
print(f'pd.cut() time:  {t_cut*1000:.2f} ms')
print(f'Speedup:        {t_apply/t_cut:.1f}x faster with vectorised pd.cut()')

# Use pd.cut() result going forward
df_clean['salary_band'] = bands_cut
print()
print('Salary band distribution:')
print(df_clean['salary_band'].value_counts().sort_index())


In [ ]:
# 3.4.4 -- Merging and joining DataFrames
#
# pd.merge() combines two DataFrames based on shared key columns.
# Analogous to SQL JOIN operations.

# Build a country metadata lookup table
country_meta = pd.DataFrame({
    'Country': ['United States', 'United Kingdom', 'Germany', 'India', 'Canada',
                'France', 'Australia', 'Brazil'],
    'region':  ['North America', 'Europe', 'Europe', 'Asia', 'North America',
                'Europe', 'Oceania', 'South America'],
    'currency': ['USD', 'GBP', 'EUR', 'INR', 'CAD', 'EUR', 'AUD', 'BRL'],
})

# Inner join -- only rows where Country exists in BOTH tables
df_merged = pd.merge(df_clean, country_meta, on='Country', how='left')
# 'left' join keeps all rows from df_clean; unmatched get NaN in meta columns

print(f'Original shape:  {df_clean.shape}')
print(f'Merged shape:    {df_merged.shape}')
print(f'New columns:     {[c for c in df_merged.columns if c not in df_clean.columns]}')

# Salary by region
if 'region' in df_merged.columns:
    region_stats = (
        df_merged.dropna(subset=['region'])
        .groupby('region')['ConvertedCompYearly']
        .agg(count='count', median='median')
        .sort_values('median', ascending=False)
        .round(0)
    )
    print()
    print('Salary by region:')
    print(region_stats.to_string())


---

## Section 3.4.5 -- Project: Complete SO 2025 Cleaning Summary

We now have a fully cleaned, analysis-ready DataFrame.
This cell produces the definitive summary that carries into Chapter 4.


In [ ]:
print('=' * 60)
print('  SO 2025 Developer Survey -- Cleaned Dataset Summary')
print('=' * 60)

print(f'Rows:                    {len(df_clean):,}')
print(f'Columns:                 {df_clean.shape[1]}')
print(f'Missing values:          {df_clean.isnull().sum().sum()}')

sal = df_clean['ConvertedCompYearly']
print(f'Salary range:            ${sal.min():,.0f} to ${sal.max():,.0f}')
print(f'Salary median:           ${sal.median():,.0f}')
print(f'Salary mean:             ${sal.mean():,.0f}')

if 'Country' in df_clean.columns:
    print(f'Countries represented:   {df_clean["Country"].nunique()}')

if 'uses_python' in df_clean.columns:
    print(f'Python adoption:         {df_clean["uses_python"].mean()*100:.1f}%')

if 'salary_band' in df_clean.columns:
    print()
    print('Salary band breakdown:')
    for band, count in df_clean['salary_band'].value_counts().sort_index().items():
        pct = count / len(df_clean) * 100
        print(f'  {str(band):<12} {count:>5,}  ({pct:.1f}%)')

print()
print('This cleaned DataFrame is the foundation for:')
print('  Chapter 4: visualisation and EDA')
print('  Chapter 5: statistical analysis')
print('  Chapter 6: machine learning pipeline')
print('  Chapter 7: deep learning')
print('  Chapter 8: NLP on text columns')


---

## Concept Check Questions

> Test your understanding before moving on. Answer each question without referring back to the notebook, then expand to check.

**Q1.** What does NumPy **broadcasting** mean? Give an example with shapes `(3,)` and `(4, 3)`.

<details><summary>Show answer</summary>

Broadcasting is NumPy's rule for operating on arrays of different shapes without copying data. A `(3,)` array is treated as `(1, 3)` and replicated across 4 rows to match `(4, 3)`. ```python
a = np.array([10, 20, 30])  # shape (3,)
b = np.ones((4, 3))         # shape (4, 3)
result = a + b              # shape (4, 3)
```

</details>

**Q2.** What is the difference between `df.loc[0]` and `df.iloc[0]`?

<details><summary>Show answer</summary>

`df.loc[0]` selects by **label** — returns the row whose *index label* is `0`. `df.iloc[0]` selects by **integer position** — always returns the first row regardless of index labels. The difference matters when the index is not a default 0-based integer range.

</details>

**Q3.** Why use `fillna(median)` rather than `fillna(mean)` for skewed salary data?

<details><summary>Show answer</summary>

The mean is pulled upward by outliers, making it unrepresentative. The median is robust to outliers and better represents the typical value. Always check the distribution's skewness before choosing an imputation strategy.

</details>

**Q4.** What does `df.groupby('Country')['Salary'].agg(['mean', 'count'])` return, and what would you check before trusting the means?

<details><summary>Show answer</summary>

A DataFrame with one row per country with `mean` and `count` columns. Check `count` — countries with n < 30 will have unreliable means. Filter to countries with sufficient sample size before drawing conclusions.

</details>

**Q5.** Explain the difference between `pd.merge(..., how='inner')` and `how='left'`.

<details><summary>Show answer</summary>

`'inner'` keeps only rows where the key exists in **both** DataFrames. `'left'` keeps all rows from the left DataFrame and fills with `NaN` where no match exists in the right. Use `'left'` when the left table is authoritative and you want to enrich it with optional right-table data.

</details>



---

## Coding Exercises

> Three exercises per chapter: **🔧 Guided** (fill-in-the-blanks) · **🔨 Applied** (write from scratch) · **🏗️ Extension** (go beyond the chapter)

Exercises use the SO 2025 developer survey dataset.
Expand each **Solution** block only after attempting the exercise.


### Exercise 1 🔧 Guided — Vectorised salary normalisation

Complete `normalise_salaries(arr: np.ndarray, method: str) -> np.ndarray`
supporting three methods:
- `'minmax'`: scale to [0, 1]
- `'zscore'`: zero mean, unit variance
- `'log'`: natural log transform (handle zeros with +1)

No loops allowed — use only NumPy vectorised operations.


In [ ]:
import numpy as np

def normalise_salaries(arr: np.ndarray, method: str) -> np.ndarray:
    if method == 'minmax':
        return  # YOUR CODE
    elif method == 'zscore':
        return  # YOUR CODE
    elif method == 'log':
        return  # YOUR CODE
    else:
        raise ValueError(f'Unknown method: {method}')

salaries = np.array([30000, 55000, 90000, 120000, 200000, 0], dtype=float)
for m in ['minmax', 'zscore', 'log']:
    print(f'{m:>8}: {normalise_salaries(salaries, m).round(3)}')

<details><summary>💡 Hint</summary>

minmax: `(arr - arr.min()) / (arr.max() - arr.min())`
zscore: `(arr - arr.mean()) / arr.std()`
log: `np.log1p(arr)` — equivalent to `log(arr + 1)`, safe for zeros

</details>

<details><summary>✅ Solution</summary>

```python
def normalise_salaries(arr, method):
    if method == 'minmax': return (arr - arr.min()) / (arr.max() - arr.min())
    elif method == 'zscore': return (arr - arr.mean()) / arr.std()
    elif method == 'log': return np.log1p(arr)
    else: raise ValueError(f'Unknown: {method}')
```

</details>


### Exercise 2 🔨 Applied — SO 2025 salary cohort analysis

Using the synthetic SO 2025 DataFrame, compute a cohort table showing,
for each (`uses_python`, `uses_ai`) combination (4 groups):
- count, median salary, 25th/75th percentile, and fraction earning > $100k

Use only pandas `groupby`, `agg`, and `apply` — no loops.


In [ ]:
import pandas as pd
import numpy as np

rng = np.random.default_rng(42)
n = 3000
df = pd.DataFrame({
    'uses_python': rng.integers(0,2,n),
    'uses_ai':     rng.integers(0,2,n),
    'salary':      np.exp(10.8 + 0.3*rng.integers(0,2,n) + rng.normal(0,0.5,n))
})

# YOUR CODE: build cohort_table
cohort_table = None
print(cohort_table)

<details><summary>💡 Hint</summary>

Use `groupby(['uses_python','uses_ai'])['salary'].agg(...)` with a dict of
named aggregations. For the `>100k` fraction, use `lambda s: (s > 100000).mean()`.

</details>

<details><summary>✅ Solution</summary>

```python
cohort_table = df.groupby(['uses_python','uses_ai'])['salary'].agg(
    count='count',
    median='median',
    p25=lambda s: s.quantile(0.25),
    p75=lambda s: s.quantile(0.75),
    pct_high_earner=lambda s: (s > 100000).mean()
).round(2)
print(cohort_table)
```

</details>


### Exercise 3 🏗️ Extension — Rolling salary percentile tracker

Write a function that, given a DataFrame with a `survey_year` column,
computes a *rolling* 3-year window percentile table showing how the
25th, 50th, and 75th salary percentiles changed year over year.

Then plot the result as a band chart (shaded area between p25 and p75,
line for p50) using matplotlib.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

rng = np.random.default_rng(42)
years = list(range(2018, 2026))
records = []
for yr in years:
    trend = (yr - 2018) * 3000
    n = 300
    records.append(pd.DataFrame({'year': yr,
        'salary': np.exp(10.8 + rng.normal(0, 0.5, n)) + trend}))
df_time = pd.concat(records, ignore_index=True)

def rolling_percentiles(df: pd.DataFrame, window: int = 3) -> pd.DataFrame:
    # YOUR CODE: return DataFrame with columns year, p25, p50, p75
    pass

result = rolling_percentiles(df_time)
print(result)
# Add your band chart here

<details><summary>💡 Hint</summary>

Use `df.groupby('year')['salary'].quantile([0.25,0.50,0.75]).unstack()` first,
then `df_pct.rolling(window).mean()` on the result.
For the band chart: `ax.fill_between(years, p25, p75, alpha=0.3)` + `ax.plot(years, p50)`

</details>

<details><summary>✅ Solution</summary>

```python
def rolling_percentiles(df, window=3):
    pcts = df.groupby('year')['salary'].quantile([0.25,0.5,0.75]).unstack()
    pcts.columns = ['p25','p50','p75']
    return pcts.rolling(window, min_periods=1).mean().reset_index()
```

</details>


---

## Chapter 3 Summary

Chapter 3 is the data science foundation every subsequent chapter builds on.

### Key Takeaways

- **NumPy arrays** store data in contiguous memory with a fixed dtype -- this is why
  operations are 100x faster than Python lists. Always check `.shape` before arithmetic.
- **Broadcasting** eliminates explicit loops: `array - array.mean()` centres every element
  without writing a single for-loop.
- **DataFrames** are labelled 2D tables. Each column is a Series with its own dtype.
- **`.loc` vs `.iloc`**: loc uses labels, iloc uses integer positions. Mixing them up
  causes silent errors. When in doubt, use `.loc`.
- **Boolean indexing** is the standard way to filter rows:
  `df[df['salary'] > 100_000]` is idiomatic Pandas.
- **Missing values** come in three forms: drop (target column), fill with constant
  (categorical), fill with median (numeric).
- **Outlier handling** with IQR bounds is more robust than mean +/- 3 std.
- **Multi-value columns** (semicolon-separated) need `.str.split().explode()` to analyse.
- **Vectorised operations** (`pd.cut`, `.str.contains`) are always faster than `.apply()`.
- **`groupby().agg()`** is the Split-Apply-Combine pattern -- equivalent to SQL GROUP BY.

### Project Thread Status

| Task | Status |
|------|--------|
| Loaded full 15,000-row SO 2025 dataset with `pd.read_csv()` | Done |
| Audited and handled missing values | Done |
| Removed salary outliers with IQR method | Done |
| Exploded multi-value language column | Done |
| Added `uses_python` and `salary_band` convenience columns | Done |
| Computed salary statistics by country, education, remote work | Done |
| Produced `df_clean` -- the analysis-ready DataFrame | Done |

---

### What's Next: Chapter 4 -- Data Visualisation

Chapter 4 takes `df_clean` and builds a full exploratory visualisation suite:
salary histograms with log scale, box plots by developer type, top-15 language
bar charts, AI tool adoption heatmaps, and interactive Plotly charts.
Every chart tells a story about the 2025 developer landscape.

---

*End of Chapter 3 -- Python for AI/ML*  
[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)
